In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib
import pandas as pd
from sense_hat import SenseHat
from collections import OrderedDict, deque, Counter
import time

__Define Colours__

In [2]:
n = [0,0,0]
g = [0,255,0]
r = [255,0,0]
b = [0,0,255]
o = [255,127,0]
n = [0,0,0]
g = [0,255,0]
b = [0,0,255]
o = [255,127,0]

__Define Shapes__

In [3]:
fhg_shape = [
n,n,n,n,n,n,n,n,
n,n,n,n,n,n,n,n,
n,n,n,n,n,n,n,n,
n,n,n,n,n,n,n,n,
n,n,n,n,g,g,g,g,
b,b,g,g,g,o,o,g,
n,n,n,n,g,g,g,g,
n,n,n,n,n,n,n,n
]

fhv_shape = [
n,n,n,n,n,n,n,n,
n,n,n,n,g,g,g,g,
b,b,g,g,g,o,o,g,
n,n,n,n,g,g,g,g,
n,n,n,n,n,n,n,n,
n,n,n,n,n,n,n,n,
n,n,n,n,n,n,n,n,
n,n,n,n,n,n,n,n
]

bhg_shape = [
n,n,n,n,n,n,n,n,
n,n,n,n,n,n,n,n,
n,n,n,n,n,n,n,n,
n,n,n,n,n,n,n,n,
g,g,g,g,n,n,n,n,
g,o,o,g,g,g,b,b,
g,g,g,g,n,n,n,n,
n,n,n,n,n,n,n,n
]

bhv_shape = [
n,n,n,n,n,n,n,n,
g,g,g,g,n,n,n,n,
g,o,o,g,g,g,b,b,
g,g,g,g,n,n,n,n,
n,n,n,n,n,n,n,n,
n,n,n,n,n,n,n,n,
n,n,n,n,n,n,n,n,
n,n,n,n,n,n,n,n
]

ser1_shape = [
n,n,n,n,g,g,g,n,
n,n,n,n,g,o,g,n,
n,n,n,n,g,o,g,n,
n,n,n,n,g,g,g,n,
n,n,n,n,n,g,n,n,
n,n,n,n,n,g,n,n,
n,n,n,n,n,b,n,n,
n,n,n,n,n,b,n,n
]

ser2_shape = [
g,g,g,n,n,g,g,g,
g,o,g,n,n,g,o,g,
g,o,g,n,n,g,o,g,
g,g,g,n,n,g,g,g,
n,g,n,n,n,n,g,n,
n,g,n,n,n,n,g,n,
n,b,n,n,n,n,b,n,
n,b,n,n,n,n,b,n
]

sma_shape = [
n,n,g,g,g,g,n,n,
n,g,o,o,o,o,g,n,
n,g,o,o,o,o,g,n,
n,n,g,g,g,g,n,n,
n,n,n,g,g,n,n,n,
n,n,n,g,g,n,n,n,
n,n,n,b,b,n,n,n,
n,n,n,b,b,n,n,n
]

qm_shape = [
n,n,n,r,r,r,n,n,
n,n,r,n,n,n,r,n,
n,n,r,n,n,n,r,n,
n,n,n,n,n,r,n,n,
n,n,n,n,r,n,n,n,
n,n,n,n,r,n,n,n,
n,n,n,n,n,n,n,n,
n,n,n,n,r,n,n,n
]

all_shapes = OrderedDict([("FHG", fhg_shape),
                          ("FHV", fhv_shape),
                          ("BHG", bhg_shape),
                          ("BHV", bhv_shape),
                          ("sv1", ser1_shape),
                          ("sv2", ser2_shape),
                          ("sma", sma_shape),
                          ("noshot", qm_shape)])

__Initialise Objects and Variables__

In [24]:
sense = SenseHat()
sense.set_imu_config(False, True, True) # Switch off Magnenometer
sense.low_light = True
sense.set_pixels(qm_shape) # Display question mark

sampling_rate = 1.0/10.0 # 0.1 second most accurate
predict_mode = True
clf = joblib.load('prod_model.pkl')
last_prediction = 'noshot'
d = deque(maxlen=7)
d.append('noshot')

In [25]:
while predict_mode:
    time_stamp = time.time()
    gyro = sense.get_gyroscope_raw()
    acceleration = sense.get_accelerometer_raw()
    
    cur_prediction = clf.predict(pd.DataFrame([[gyro['x'],gyro['y'],gyro['z'],acceleration['x'],acceleration['y'],acceleration['z']]]))
    cur_pred_prob = max(max(clf.predict_proba(pd.DataFrame([[gyro['x'],gyro['y'],gyro['z'],acceleration['x'],acceleration['y'],acceleration['z']]]))))
    
    if (cur_pred_prob > .55):
        d.append(cur_prediction[0])
    
    sense.set_pixels(all_shapes.get(Counter(list(d)).most_common(1)[0][0]))
    
    while time.time() <= time_stamp + sampling_rate:
        pass
    
    for event in sense.stick.get_events():
        if event.action == "released":
            predict_mode = False